In [1]:
import numpy as np
import pandas as pd 

In [2]:
temp_df = pd.read_csv('IMDB Dataset.csv')

In [3]:
df = temp_df.iloc[:10000]

In [4]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
df.drop_duplicates(inplace=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_20492\3006716147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [6]:
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text

In [7]:
df['review'] = df['review'].apply(remove_tags)

C:\Users\HP\AppData\Local\Temp\ipykernel_20492\2336150696.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(remove_tags)


In [8]:
df['review'] = df['review'].apply(lambda x:x.lower())

C:\Users\HP\AppData\Local\Temp\ipykernel_20492\740760900.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x:x.lower())


In [9]:
from nltk.corpus import stopwords

sw_list = stopwords.words('english')

df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

C:\Users\HP\AppData\Local\Temp\ipykernel_20492\2826946130.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))


In [10]:
df['review']


0       one reviewers mentioned watching 1 oz episode ...
1       wonderful little production. filming technique...
2       thought wonderful way spend time hot summer we...
3       basically there's family little boy (jake) thi...
4       petter mattei's "love time money" visually stu...
                              ...                        
9995    fun, entertaining movie wwii german spy (julie...
9996    give break. anyone say "good hockey movie"? kn...
9997    movie bad movie. watching endless series bad h...
9998    movie probably made entertain middle school, e...
9999    smashing film film-making. shows intense stran...
Name: review, Length: 9983, dtype: object

In [11]:
import gensim

In [12]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [13]:
story = []
for doc in df['review']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [14]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [15]:
model.build_vocab(story)


In [16]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(5875860, 6212140)

In [17]:
len(model.wv.index_to_key)

31845

In [18]:
def document_vector(doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [19]:
document_vector(df['review'].values[0])

array([-0.19637977,  0.47149754,  0.11812351,  0.24257267, -0.14163001,
       -0.64171267,  0.24937868,  0.9610583 , -0.31169897, -0.2843822 ,
       -0.31329504, -0.4941673 ,  0.09871315,  0.1273734 ,  0.20418206,
       -0.07803781, -0.02926635, -0.37474802, -0.11689337, -0.61341876,
        0.02389806,  0.27066123,  0.08482017, -0.29432464, -0.30133158,
       -0.04454774, -0.31860015, -0.01398919, -0.3342366 ,  0.02512546,
        0.3128859 , -0.01975483,  0.16675065, -0.24141648, -0.12015162,
        0.5084522 ,  0.11305447, -0.40602732, -0.2657407 , -0.71685344,
        0.09500193, -0.28172958,  0.07758101, -0.08587583,  0.4883739 ,
       -0.11846142, -0.24867404, -0.04243556,  0.12566462,  0.34278545,
        0.06265073, -0.37843722, -0.4167666 , -0.07654494, -0.09648264,
        0.21693784,  0.2321294 , -0.00500608, -0.28903285,  0.1109437 ,
        0.0776165 ,  0.09723758, -0.00534982, -0.09687065, -0.41960377,
        0.25329816,  0.05584517,  0.0993025 , -0.295234  ,  0.28

In [20]:
from tqdm import tqdm

In [21]:
X = []
for doc in tqdm(df['review'].values):
    X.append(document_vector(doc))

100%|██████████████████████████████████████████████████████████████████████████████| 9983/9983 [19:20<00:00,  8.60it/s]


In [22]:
X = np.array(X)

In [23]:
X[0]

array([-0.19637977,  0.47149754,  0.11812351,  0.24257267, -0.14163001,
       -0.64171267,  0.24937868,  0.9610583 , -0.31169897, -0.2843822 ,
       -0.31329504, -0.4941673 ,  0.09871315,  0.1273734 ,  0.20418206,
       -0.07803781, -0.02926635, -0.37474802, -0.11689337, -0.61341876,
        0.02389806,  0.27066123,  0.08482017, -0.29432464, -0.30133158,
       -0.04454774, -0.31860015, -0.01398919, -0.3342366 ,  0.02512546,
        0.3128859 , -0.01975483,  0.16675065, -0.24141648, -0.12015162,
        0.5084522 ,  0.11305447, -0.40602732, -0.2657407 , -0.71685344,
        0.09500193, -0.28172958,  0.07758101, -0.08587583,  0.4883739 ,
       -0.11846142, -0.24867404, -0.04243556,  0.12566462,  0.34278545,
        0.06265073, -0.37843722, -0.4167666 , -0.07654494, -0.09648264,
        0.21693784,  0.2321294 , -0.00500608, -0.28903285,  0.1109437 ,
        0.0776165 ,  0.09723758, -0.00534982, -0.09687065, -0.41960377,
        0.25329816,  0.05584517,  0.0993025 , -0.295234  ,  0.28

In [24]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

y = encoder.fit_transform(df['sentiment'])

In [25]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [26]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [28]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7801702553830746